In [13]:
import pandas as pd
import re

def create_unique_id_generator():
    """Функция для создания глобального генератора."""
    counter = 1
    while True:
        yield counter
        counter += 1

def get_ids_for_empty_name(value, id_generator):
    if value == "":
        return f"anon_{next(id_generator)}"
    else:
        return value

def transform_work_list(work_list):
    work_list = work_list.strip("[]")
    parsed_list = [item.strip() for item in work_list.split(",")]
    сompanies_num = parsed_list.count("Компания")
    freelance_num = parsed_list.count("Фриланс")
    total_num = сompanies_num + freelance_num
    works_info = [
        {
            "name": parsed_list[total_num + i],
            "post": parsed_list[total_num * 2 + i],
            "salary": parsed_list[total_num * 3 + i]
        } if total_num * 3 + i < len(parsed_list) and parsed_list[total_num * 3 + i] != "" else
        {
            "name": parsed_list[total_num + i],
            "post": parsed_list[total_num * 2 + i],
            "salary": None
        }
        for i in range(total_num)
    ]
    return works_info

def get_data_from_file(file_path: str) -> pd.DataFrame:
    unique_id_generator = create_unique_id_generator()
    data = pd.read_excel(
        file_path,
        converters={
            "ФИО": lambda x: get_ids_for_empty_name(x, unique_id_generator),
        }
    )
    data["Где работал"] = data["Где работал"].map(transform_work_list)
    return data

In [14]:
data = get_data_from_file("yandex form 2.xlsx")

In [15]:
data

,Дата,ФИО,Год выпуска,Матпоток,Магистратура,Где работал,Сферы работы,Другие сферы
0,2024-09-09 22:58:28,Евглевская Анна Васильевна,2019,Да,"[Нет,,]","[{'name': 'SGS Vostok Limited', 'post': 'Бизне...",Разработка прикладного программного обеспечени...,Анализ данных и визуализация
1,2024-09-09 22:47:47,Карпенко Мария Владимировна,2019,Да,"[Нет,,]","[{'name': 'Яндекс', 'post': 'Старший разработч...",Разработка прикладного программного обеспечения,NaN
2,2024-09-09 22:51:59,Рябова Светлана Олеговна,2019,Да,"[Да,New Mexico Highlands University ,Computer ...","[{'name': 'University at Buffalo', 'post': 'Ас...","Оптимизационные задачи,Задачи машинного обучения",NaN
3,2024-09-09 22:53:10,Громова Надежда (дев. Родина),2019,Да,"[Нет,,]","[{'name': 'ООО «СКТ Групп»', 'post': 'Специали...","Оптимизационные задачи,Экономический анализ",NaN
4,2024-09-09 23:32:30,Самойлов Никита Александрович,2016,Да,"[Да,СПбГЭУ,Прикладная математика и информатика]","[{'name': 'ПАО ""Газпром нефть""', 'post': 'Руко...",Разработка прикладного программного обеспечени...,NaN
...,...,...,...,...,...,...,...,...
83,2024-09-25 18:47:00,anon_20,2020,Да,"[Да,СПБГЭУ, Анализ данных в экономике]","[{'name': 'ООО Райдер Софт', 'post': 'Менеджер...","Экономический анализ, Другое",NaN
84,2024-09-25 20:46:00,anon_21,2020,Да,"[Нет,, ]","[{'name': 'ПАО ""Газпром нефть""', 'post': 'BI а...",Другое,"Анализ данных, etl процессы, построение отчетн..."
85,2024-09-26 14:16:00,anon_22,2017,Да,"[Да,СПбГЭУ, ]","[{'name': 'РСТИ', 'post': 'Аналитик', 'salary'...","Оптимизационные задачи, Экономический анализ, ...","Организация бизнес процессов, внедрение програ..."
86,2024-09-26 22:30:00,Кунякин Виктор Геннадьевич,2019,Нет,"[Нет,, ]","[{'name': 'Vattenfall Energy Trading', 'post':...",Экономический анализ,NaN


In [16]:
def parse_mean_salary(salary):
    if salary is None:
        return None
    numbers = [int(num.replace(" ", "")) for num in re.findall(r"\d+\s?\d*", salary)]
    if len(numbers) == 2:
        return sum(numbers) / 2
    elif len(numbers) == 1:
        return numbers[0]
    else:
        return None

In [17]:
def parse_magistracy(magistracy):
    magistracy = magistracy.strip("[]")
    magistracy = [item.strip() for item in magistracy.split(",")]
    return magistracy

def flatten_data(data: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for idx, row in data.iterrows():
        for step, job in enumerate(row["Где работал"]):
            rows.append({
                "graduate": idx,
                "step": step,
                "company": job["name"],
                "position": job["post"],
                "salary": job["salary"],
                "graduation_year": row["Год выпуска"],
                "math_group": row["Матпоток"],
                "magistracy": parse_magistracy(row["Магистратура"])
                # "salary_mean": parse_mean_salary(job["salary"])
            })
    data = pd.DataFrame(rows)
    # data["salary_mean"] = (
    #     data.groupby("person_id")["salary_mean"]
    #           .transform(lambda group: group.ffill().bfill())
    # )

    data = data.sort_values(by=["step"])

    # data["current_salary"] = data.groupby("person_id")["salary_interval"].transform("last")
    # data = data.sort_values("current_salary")

    data = data.sort_values(by=["graduate", "step"])

    return data

In [18]:
data = flatten_data(data)
data

,graduate,step,company,position,salary,graduation_year,math_group,magistracy
0,0,0,SGS Vostok Limited,Бизнес-аналитик,50 000 руб. — 100 000 руб.,2019,Да,"[Нет, , ]"
1,0,1,Devim,Старший бизнес-аналитик,150 000 руб. — 200 000 руб.,2019,Да,"[Нет, , ]"
2,0,2,S&P Global,Analyst,300 000 руб. — 350 000 руб.,2019,Да,"[Нет, , ]"
3,1,0,Яндекс,Старший разработчик,Более 400 000 руб.,2019,Да,"[Нет, , ]"
4,2,0,University at Buffalo,Аспирант,None,2019,Да,"[Да, New Mexico Highlands University, Computer..."
...,...,...,...,...,...,...,...,...
137,84,2,Т-банк Центр Разработки,Старший BI аналитик,200 000 руб. — 250 000 руб.,2020,Да,"[Нет, , ]"
138,85,0,РСТИ,Аналитик,200 000 руб. — 250 000 руб.,2017,Да,"[Да, СПбГЭУ, ]"
139,86,0,Vattenfall Energy Trading,Junior Options Trader,Более 400 000 руб.,2019,Нет,"[Нет, , ]"
140,87,0,Пивоваренная компания «Балтика»,Demand planer,100 000 руб. — 150 000 руб.,2018,Да,"[Да, СПбГЭУ, ]"


In [19]:
def get_magistracy(x):
    if x[1] == "СПбГЭУ" and any([(substring in x[2].lower())
    for substring in ["пм", "прикладная математика", "анализ данных"]]):
        return "Магистратура ПМ СПбГЭУ"
    elif x[0] == "Да":
        return "Другая магистратура"
    else:
        return "Без магистратуры"

In [20]:
data["magistracy"] = data["magistracy"].map(get_magistracy)

In [21]:
data.rename(
    {"salary": "salary_range"},
    axis=1,
    inplace=True
)

In [22]:
import sqlite3
cnx = sqlite3.connect('graduates.db')

In [23]:
data.to_sql(name="graduates", con=cnx, index=False)

142

In [24]:
data

,graduate,step,company,position,salary_range,graduation_year,math_group,magistracy
0,0,0,SGS Vostok Limited,Бизнес-аналитик,50 000 руб. — 100 000 руб.,2019,Да,Без магистратуры
1,0,1,Devim,Старший бизнес-аналитик,150 000 руб. — 200 000 руб.,2019,Да,Без магистратуры
2,0,2,S&P Global,Analyst,300 000 руб. — 350 000 руб.,2019,Да,Без магистратуры
3,1,0,Яндекс,Старший разработчик,Более 400 000 руб.,2019,Да,Без магистратуры
4,2,0,University at Buffalo,Аспирант,None,2019,Да,Другая магистратура
...,...,...,...,...,...,...,...,...
137,84,2,Т-банк Центр Разработки,Старший BI аналитик,200 000 руб. — 250 000 руб.,2020,Да,Без магистратуры
138,85,0,РСТИ,Аналитик,200 000 руб. — 250 000 руб.,2017,Да,Другая магистратура
139,86,0,Vattenfall Energy Trading,Junior Options Trader,Более 400 000 руб.,2019,Нет,Без магистратуры
140,87,0,Пивоваренная компания «Балтика»,Demand planer,100 000 руб. — 150 000 руб.,2018,Да,Другая магистратура


In [98]:
data.groupby(["graduation_year", "company"]).agg(agg_data=("graduate", "count"))

agg_data
graduation_year company                        
2016                                          2
                АК «АЛРОСА» (ПАО)             1
                ПАО "Газпром нефть"           2
2017            AdVenture                     1
                AppQuantum                    1
...                                         ...
2024            Астех Индастриз               1
                Газстройпром                  1
                ООО "Газпром ВНИИГАЗ"         1
                ООО "Гротекс"                 1
                ООО "Контакт"                 1

[123 rows x 1 columns]